In [1]:
import pandas as pd
import numpy as np
import json
import pickle

from tqdm import tqdm_notebook

from datetime import datetime

In [2]:
with open('../data/user_features.json', 'r') as f:
    user_features = json.loads(f.read())
    
with open('../models/big_model_scoring', 'rb') as f:
    big_model_scoring = pickle.loads(f.read())
    
with open('../models/small_model_scoring', 'rb') as f:
    small_model_scoring = pickle.loads(f.read())

In [3]:
user_change = pd.read_csv('../data/user_change.csv')

In [4]:
user_change.head()

,user_id,change_id
0,216667066.0,5
1,216667249.0,4
2,216667667.0,4
3,216668213.0,11
4,216668899.0,15


In [5]:
input_df = {}

for key in user_features['433602817.0'].keys():
    input_df[key] = []
    
for u in user_change.user_id.values:
    for key in input_df.keys():
        input_df[key].append(user_features[str(float(u))][key])

In [7]:
input_df = pd.DataFrame(input_df)

In [8]:
input_df.head()

,exp_years,age,widow,no_mirrage,divorced,of_mirrage,unof_mirrage,lone_mother,good_job,holiday,...,mean_good_job,mean_holiday,mean_motivation,mean_prize_1,mean_prize_2,mean_prize_3,mean_prize_4,mean_prize_5,mean_prize_6,mean_prize_7
0,19.416667,55.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.015529,0.004495,0.0,0.0,0.000409,0.0,0.0,0.0,0.0,0.0
1,18.416667,48.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.015529,0.004495,0.0,0.0,0.000409,0.0,0.0,0.0,0.0,0.0
2,18.416667,47.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.015529,0.004495,0.0,0.0,0.000409,0.0,0.0,0.0,0.0,0.0
3,19.416667,49.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.015529,0.004495,0.0,0.0,0.000409,0.0,0.0,0.0,0.0,0.0
4,18.416667,46.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.015529,0.004495,0.0,0.0,0.000409,0.0,0.0,0.0,0.0,0.0


In [14]:
input_df.shape

(4138, 56)

In [9]:
scores_1 = [p[1] for p in big_model_scoring.predict_proba(input_df)]

In [24]:
first_score = {}

for i in range(len(scores_1)):
    first_score[user_change.user_id.values[i]] = scores_1[i]

In [17]:
normal_columns = [
    'exp_years',
    'good_job',
    'holiday',
    'motivation',
    'prize_1',
    'prize_2',
    'prize_3',
    'prize_4',
    'prize_5',
    'prize_6',
    'prize_7'
]

new_normal_data = {}

for c in normal_columns:
    new_normal_data[c] = input_df[c].values
    
new_normal_data = pd.DataFrame(new_normal_data)

In [18]:
scores_2 = [p[1] for p in small_model_scoring.predict_proba(new_normal_data)]

In [25]:
second_score = {}

for i in range(len(scores_2)):
    second_score[user_change.user_id.values[i]] = scores_2[i]

In [28]:
change_user_scores = {}

for change, group in tqdm_notebook(user_change.groupby('change_id')):
    change_user_scores[change] = {}
    change_user_scores[change]['users'] = group.user_id.values.tolist()
    change_user_scores[change]['big_scoring'] = [first_score[u] for u in group.user_id.values.tolist()]
    change_user_scores[change]['small_scoring'] = [second_score[u] for u in group.user_id.values.tolist()]

HBox(children=(IntProgress(value=0, max=14), HTML(value='')))

In [30]:
with open('../data/answer_scores_for_change.json', 'w') as f:
    f.write(json.dumps(change_user_scores))